## Methods analysis for solving the problem statement and stating the hypothesis for, What is duplicate images?

## Hypothesis: 
   #### any image having same pixel values or features, that of query image is said to be a duplicate image

#### Importing all the required libraries

In [10]:
import cv2
import numpy as np
from scipy.spatial import distance
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import Model

import os

#### Method 1:Approch uses standerd Image Processig technique.For comparing two images simply subtract them and check the image difference matrix it it contains value.

In [2]:
def resize(image, size=(250,250)):
    return cv2.resize(image,size)

def sub_similarity(image1, image2):
    image1 = resize(image1)
    image2 = resize(image2)
    difference = cv2.subtract(image1, image2)
    result = not np.any(difference)
    if result is True:
        print("same")
        return True
    else:
        print("different")
        return False

In [4]:
a = cv2.imread("./my_try/my_try0_1.jpg")
b = cv2.imread("./my_try/my_try0_1.jpg")
c = cv2.imread("./my_try/my_try1_1.jpg")
status_a_b = sub_similarity(a,b)
print(status_a_b)
print("************")
status_b_c = sub_similarity(b,c)
print(status_b_c)

same
True
************
different
False


#### The above method is good if the required case is such that, we have to compare only one pair of image at a time.But in this method chances of faliure is high. As we are only considering pixel values  for the calculations.

******************************************************************************

#### Method 2: Using the mean square error (mse). As the images are nothing but the two dimensional signal. Therefore, signal processing technique can also ne used to check the similarity

In [5]:
def mse(image1, image2):
    image1 = resize(image1)
    image2 = resize(image2)
    error = np.sum((image1.astype("float") - image2.astype("float")) ** 2)
    return error

In [7]:
a = cv2.imread("./my_try/my_try0_1.jpg")
b = cv2.imread("./my_try/my_try0_1.jpg")
c = cv2.imread("./my_try/my_try1_1.jpg")
status_a_b = mse(a,b)
status_b_c = mse(b,c)
print("The mse between a and b is {:.2f}".format(status_a_b))
print("The mse between b and c is {:.2f}".format(status_b_c))

The mse between a and b is 0.00
The mse between b and c is 463947575.00


#### The method of mse is so not going to solve the problem for us as the requirment, as mse tends to provide the output in analog way where 0 means similar and away from 0 towards + inf is different. Although values can be  normalized to stay in range on 0 to 1. But its not very ideal way.

***********************************************

#### Method 3: use of deep learning where a pre-trained model is used to extract the features and then clustering them  to store the like-ones together so when a query image comes, just extract fetures and look for cluster which is closest to query image fetures. 

In [8]:
def get_model():
    base_model = VGG16(weights='imagenet', include_top=True)
    model = Model(inputs=base_model.input,
                  outputs=base_model.get_layer('fc2').output)
    return model

def feature_extraction(img,model):
    img = cv2.resize(img,(224,224))
    img = image.img_to_array(img)
    img4d = np.expand_dims(img, axis=0)
    pred = model.predict(img4d)[0,:]
    return pred



In [9]:
model = get_model()
a = cv2.imread("./my_try/my_try0_1.jpg")
b = cv2.imread("./my_try/my_try0_1.jpg")
c = cv2.imread("./my_try/my_try1_1.jpg")
f_a = feature_extraction(a, model)
f_b = feature_extraction(b, model)
f_c = feature_extraction(c, model)

del model

In [11]:

d_a_b = distance.euclidean(f_a, f_b)
d_b_c = distance.euclidean(f_b, f_c)

print("A to B image distance is {}\n and B to C image distance is {:.2f}".format(d_a_b, d_b_c))

A to B image distance is 0.0
 and B to C image distance is 44.21


#### As we can see that with deep learning method also we can check the similarity of two images. Here we can see that similar images tends to have zero spatial distance between them while non-similar have mre distance between

************************

#### Before finalizing the approch to be used for solving the problem, here are two more methods that can be considered for this task. But this are not suitable for genric use as according to me their final values are highly susceptible to noise. 

#### 1) SSIM : structural similarity index measurment 
#### 2) Hash_signature : calculating the hash value of the image.

************

#### So to finalize deep learning approch(Method 3) will be used as each image can be preprocessed and can be stored as mat file which is faster way to access the data when we need to load images regularly. and images can be represented by smaller array.